In [1]:
# samples
from collections import deque

# read data & 
with open('data/day07_sample.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

In [2]:
# Part 1: Find sizes of each
dir_dict = {}
for i in range(len(data)):
    if data[i] == '$ cd ..':
        continue
    elif data[i].startswith('$ cd'):
        v = data[i].split('$ cd ')[1]
        dir_dict[v] = []
        try:
            i += 2 # move ahead past LS command
            next_step = data[i]
            while not next_step.startswith('$'):
                if next_step.startswith('dir'):
                    dir_dict[v].append(next_step.split('dir ')[1])
                else:
                    dir_dict[v].append(int(next_step.split(' ')[0]))
                i += 1
                next_step = data[i]
        except:
            continue
    else:
        continue

In [3]:
dir_dict

{'/': ['a', 14848514, 8504156, 'd'],
 'a': ['e', 29116, 2557, 62596],
 'e': [584],
 'd': [4060174, 8033020, 5626152, 7214296]}

In [4]:
# and now recursion
def recursionSum(d, k):
    """Iterate over values in key, adding counts)"""
    sum_v = 0
    
    # iterate over each value of colors
    for c in d[k]:
        if c in d.keys():
            sum_v = (sum_v + recursionSum(d, c))
        else:
            sum_v += c
    return sum_v

In [5]:
final_sums = 0

for k in dir_dict.keys():
    out = recursionSum(dir_dict, k)
    if out <= 100_000:
        final_sums += out

print(final_sums)

95437


In [6]:
import random 

with open('data/day07.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

# Part 1: Find sizes of each
dir_dict = {}
for i in range(len(data)):
    if data[i] == '$ cd ..':
        continue
    elif data[i].startswith('$ cd'):
        v = data[i].split('$ cd ')[1]
        dir_dict[v] = []
        try:
            i += 2 # move ahead past LS command
            next_step = data[i]
            while not next_step.startswith('$'):
                if next_step.startswith('dir'):
                    dir_dict[v].append(next_step.split('dir ')[1])
                else:
                    dir_dict[v].append(int(next_step.split(' ')[0]))
                i += 1
                next_step = data[i]
        except:
            continue
    else:
        continue

In [7]:
# manual sanity checks
assert(dir_dict['wvdlv'] == [314303])
assert(dir_dict['fjmbvhh'] == [244287, 146029])
assert(dir_dict['pjmc'] == [106609]) # ah shoot, i need to actually keep the full directory names (not just ignore the "..")


AssertionError: 

### Early, Painful Finding:

- As shown above, my manual value of `106609` did not match with my dictionary for key `pjmc`. This is because there are multiple directories called `pjmc` that have different full paths.
- To handle this, below I use a `deque` to keep tabs on where I am in the path in order to store distinct versions of directories when needed. This was painful!
- Note: the `///` instead of `/` doesn't matter as long as consistent.

In [8]:
# Part 1: Find sizes of each
dir_dict = {}
# read data & 
with open('data/day07_sample.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

for i in range(len(data)):
    if data[i] == '$ cd /':
        path_deque = deque(['/'])
        
    if data[i] == '$ cd ..':
        path_deque.pop()
    elif data[i].startswith('$ cd'):
        v = data[i].split('$ cd ')[1]
        #print(v)
        path_deque.append(f'{v}/')
        #print(path_deque)
        path_key = ''.join([x for x in path_deque])
        dir_dict[path_key] = []
        try:
            i += 2 # move ahead past LS command
            next_step = data[i]
            while not next_step.startswith('$'):
                if next_step.startswith('dir'):
                    v = next_step.split('dir ')[1]
                    dir_dict[path_key].append(f'{path_key}{v}/')
                else:
                    dir_dict[path_key].append(int(next_step.split(' ')[0]))
                i += 1
                next_step = data[i]
        except:
            continue
    else:
        continue
dir_dict

{'///': ['///a/', 14848514, 8504156, '///d/'],
 '///a/': ['///a/e/', 29116, 2557, 62596],
 '///a/e/': [584],
 '///d/': [4060174, 8033020, 5626152, 7214296]}

In [9]:
final_sums = 0

for k in dir_dict.keys():
    out = recursionSum(dir_dict, k)
    if out <= 100_000:
        final_sums += out

print(final_sums)

95437


In [10]:
# actual data
with open('data/day07.txt') as fh:
    data = [line.strip() for line in fh.readlines()]
    
# Part 1: Find sizes of each
dir_dict = {}

for i in range(len(data)):
    if data[i] == '$ cd /':
        path_deque = deque(['/'])
        
    if data[i] == '$ cd ..':
        path_deque.pop()
    elif data[i].startswith('$ cd'):
        v = data[i].split('$ cd ')[1]
        path_deque.append(f'{v}/')
        path_key = ''.join([x for x in path_deque])
        dir_dict[path_key] = []
        try:
            i += 2 # move ahead past LS command
            next_step = data[i]
            while not next_step.startswith('$'):
                if next_step.startswith('dir'):
                    v = next_step.split('dir ')[1]
                    dir_dict[path_key].append(f'{path_key}{v}/')
                else:
                    dir_dict[path_key].append(int(next_step.split(' ')[0]))
                i += 1
                next_step = data[i]
        except:
            continue
    else:
        continue
            
final_sums = 0
final_sum_dict = {}
for k in dir_dict.keys():
    out = recursionSum(dir_dict, k)
    final_sum_dict[k] = out
    if out <= 100_000:
        final_sums += out
        

print(final_sums)

1501149


In [11]:
### Part 2: Thank you for going easier on us
total = 70000000
need = 30000000
used = final_sum_dict['///']
unused = total - used
remaining_need = need - unused
print(f"We need to delete at least: {remaining_need}")

min_v = 999_999_999
for k,v in final_sum_dict.items():
    if v >= remaining_need:
        if v < min_v:
            min_v = v
            print(f"In path {k} found space of {v}")
print(min_v)

We need to delete at least: 9199225
In path /// found space of 49199225
In path ///gwnwqcgq/ found space of 44484503
In path ///gwnwqcgq/mdhln/ found space of 22685292
In path ///gwnwqcgq/mdhln/jhmvgjrr/ found space of 10096985
10096985
